In [1]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectPercentile, VarianceThreshold, f_classif
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.preprocessing import FunctionTransformer
from copy import copy
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

In [2]:
train_path = '../data/hy_round2_train_20200225'

In [3]:
def feature_generate_manually():
    train_df_list = []
    for file_name in os.listdir(train_path):
        df = pd.read_csv(os.path.join(train_path, file_name))
        train_df_list.append(df)

    train_df = pd.concat(train_df_list)

    train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')

    all_df = pd.concat([train_df], sort=False)

    all_df['x'] = all_df['lat'].values
    all_df['y'] = all_df['lon'].values

    new_df = all_df.groupby('渔船ID').agg(x_min=('x', 'min'), x_max=('x', 'max'), x_mean=('x', 'mean'), x_std=('x', 'std'), x_skew=('x', 'skew'), x_sum=('x', 'sum'),
                y_min=('y', 'min'), y_max=('y', 'max'), y_mean=('y', 'mean'), y_std=('y', 'std'), y_skew=('y', 'skew'), y_sum=('y', 'sum'),
                v_min=('速度', 'min'), v_max=('速度', 'max'), v_mean=('速度', 'mean'), v_std=('速度', 'std'), v_skew=('速度', 'skew'), v_sum=('速度', 'sum'),
                d_min=('方向', 'min'), d_max=('方向', 'max'), d_mean=('方向', 'mean'), d_std=('方向', 'std'), d_skew=('方向', 'skew'), d_sum=('方向', 'sum'))
    new_df['x_max-x_min'] = new_df['x_max'] - new_df['x_min']
    new_df['y_max-y_min'] = new_df['y_max'] - new_df['y_min']
    new_df['x_max-y_min'] = new_df['x_max'] - new_df['y_min']
    new_df['y_max-x_min'] = new_df['y_max'] - new_df['x_min']

    new_df['slope'] = new_df['y_max-y_min'] / np.where(new_df['x_max-x_min']==0, 0.001, new_df['x_max-x_min'])
    new_df['area'] = new_df['x_max-x_min'] * new_df['y_max-y_min']

    xy_cov = []
    unique_x = []
    unique_x_rate = []
    for ship_id, group in all_df.groupby('渔船ID'):
        x = group['x'].values
        y = group['y'].values
        xy_cov.append(group['x'].cov(group['y']))
        unique_x.append(np.unique(x).shape[0])
        unique_x_rate.append(np.unique(y).shape[0] / x.shape[0])

    new_df['xy_cov'] = xy_cov
    new_df['unique_x'] = unique_x
    new_df['unique_x_rate'] = unique_x_rate

    new_df['type'] = all_df.groupby('渔船ID').agg(type=('type', 'first'))['type'].values

    X_train = new_df.drop(columns=['type']).iloc[:len(train_df_list)]
    y_train = new_df.iloc[:len(train_df_list)]['type']

    return X_train, y_train


def feature_generate_tsfresh():
    train_df = pd.read_csv('./feature/train.csv', index_col=0)
    X_train = train_df.drop(columns=['type'])
    y_train = train_df['type']

    base_model = lgb.LGBMClassifier(n_estimators=1000, subsample=0.8)
    base_model.fit(X_train.values, y_train)

    selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:50]]
    print(selected_columns)

    X_train = X_train[selected_columns]

    X_train_manully, _ = feature_generate_manually()

    print(X_train.shape, X_train_manully.shape)

    X_train['x_max-x_min'] = X_train_manully['x_max-x_min'].values
    X_train['x_max-y_min'] = X_train_manully['x_max-y_min'].values
    X_train['y_max-x_min'] = X_train_manully['y_max-x_min'].values
    X_train['y_max-y_min'] = X_train_manully['y_max-y_min'].values

    X_train['slope'] = X_train_manully['slope'].values
    X_train['area'] = X_train_manully['area'].values

    return X_train, y_train.values

In [4]:
def get_model():
    exported_pipeline = make_pipeline(
        SelectPercentile(score_func=f_classif, percentile=48),
        StackingEstimator(estimator=SGDClassifier(alpha=0.01, eta0=0.01, fit_intercept=False, l1_ratio=0.25, learning_rate="invscaling", loss="modified_huber", penalty="elasticnet", power_t=10.0)),
        ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.6000000000000001, min_samples_leaf=1, min_samples_split=3, n_estimators=100)
    )

    set_param_recursive(exported_pipeline.steps, 'random_state', 42)
    return exported_pipeline


def get_model_v2():
    exported_pipeline = make_pipeline(
        make_union(
            make_pipeline(
                make_union(
                    FunctionTransformer(copy),
                    FunctionTransformer(copy)
                ),
                SelectPercentile(score_func=f_classif, percentile=18)
            ),
            FunctionTransformer(copy)
        ),
        StackingEstimator(estimator=SGDClassifier(alpha=0.01, eta0=0.1, fit_intercept=False, l1_ratio=1.0, learning_rate="constant", loss="hinge", penalty="elasticnet", power_t=0.1)),
        VarianceThreshold(threshold=0.05),
        ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.55, min_samples_leaf=1, min_samples_split=4, n_estimators=100)
    )
    set_param_recursive(exported_pipeline.steps, 'random_state', 42)
    return exported_pipeline


from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler, StandardScaler
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.preprocessing import FunctionTransformer
from copy import copy

def get_model_v3():
    exported_pipeline = make_pipeline(
        make_union(
            FunctionTransformer(copy),
            FunctionTransformer(copy)
        ),
        RobustScaler(),
        RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.25, n_estimators=100), step=0.6500000000000001),
        StandardScaler(),
        GradientBoostingClassifier(learning_rate=0.5, max_depth=9, max_features=0.05, min_samples_leaf=18, min_samples_split=3, n_estimators=100, subsample=0.9000000000000001)
    )
    # Fix random state for all the steps in exported pipeline
    set_param_recursive(exported_pipeline.steps, 'random_state', 42)
    return exported_pipeline


from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator, ZeroCount
from xgboost import XGBClassifier
from tpot.export_utils import set_param_recursive

def get_model_v4():
    exported_pipeline = make_pipeline(
        StackingEstimator(estimator=XGBClassifier(learning_rate=0.001, max_depth=2, min_child_weight=17, n_estimators=100, nthread=1, subsample=0.8)),
        ZeroCount(),
        VarianceThreshold(threshold=0.2),
        RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.15000000000000002, n_estimators=100), step=0.2),
        GradientBoostingClassifier(learning_rate=0.5, max_depth=7, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=3, n_estimators=100, subsample=1.0)
    )
    # Fix random state for all the steps in exported pipeline
    set_param_recursive(exported_pipeline.steps, 'random_state', 37)
    return exported_pipeline

In [5]:
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

def get_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance


def get_feature(arr):
    feature = [np.max(arr), np.quantile(arr, 0.9), np.quantile(arr, 0.1),
               np.quantile(arr, 0.75), np.quantile(arr, 0.25), np.mean(arr), np.std(arr),
               np.median(arr),  np.std(arr) / np.mean(arr)]
    feature.append(np.corrcoef(np.array([arr[:-1], arr[1:]]))[0, 1])
    feature.append(skew(arr))
    feature.append(kurtosis(arr))
    return feature

def get_paper_feature():
    train_df_list = []
    for file_name in os.listdir(train_path):
        df = pd.read_csv(os.path.join(train_path, file_name))
        train_df_list.append(df)

    train_df = pd.concat(train_df_list)

    train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')

    all_df = pd.concat([train_df], sort=False)

    features = []
    for ship_id, group in all_df.groupby('渔船ID'):

        type_ = group['type'].values[0]

        group = group.sort_values(by=['time'])
        lat = group['lat'].values
        lon = group['lon'].values
        time_ = group['time'].values
        dire = group['方向'].values

        speed_list = []
        for i in range(lat.shape[0]):
            if i == 0:
                continue
            hour = (time_[i] - time_[i-1]) / np.timedelta64(1,'h')
            dist = geodesic((lat[i-1], lon[i-1]), (lat[i], lon[i])).km
            speed_list.append(dist / hour)

#         acc_list = []
#         for i in range(len(speed_list)):
#             if i == 0:
#                 continue
#             hour = (time_[i] - time_[i-1]) / np.timedelta64(1,'h')
#             acc = (speed_list[i] - speed_list[i-1]) / hour
#             acc_list.append(acc)

        c = np.sum(np.cos(dire / 180 * np.pi)) / group.shape[0]
        s = np.sum(np.sin(dire / 180 * np.pi)) / group.shape[0]
        r = np.sqrt(c ** 2 + s ** 2)
        theta = np.arctan(s / c)
        angle_feature = [r, theta, np.sqrt(-2 * np.log(r))]
        
        turn_list = []
        for i in range(dire.shape[0]):
            if i == 0:
                continue
            turn = 1 - np.cos(dire[i-1] / 180 * np.pi - dire[i] / 180 * np.pi)
            turn_list.append(turn * np.pi)
        turn_list = np.array(turn_list)
        c = np.sum(np.cos(turn_list)) / (group.shape[0] - 1)
        s = np.sum(np.sin(turn_list)) / (group.shape[0] - 1)
        r = np.sqrt(c ** 2 + s ** 2)
        theta = np.arctan(s / c)
        turn_feature =  [r, theta, np.sqrt(-2 * np.log(r))]
        
#         sinuosity = []
#         length = 2
#         for i in range(lat.shape[0]):
#             if i <= length-1:
#                 continue
#             dist_line = get_distance(lat[i-length], lon[i-length], lat[i], lon[i])

#             dist_sum = 0
#             for j in range(length):
#                 dist_sum += get_distance(lat[i-j-1], lon[i-j-1], lat[i-j], lon[i-j])

#             if dist_line == 0:
#                 sinuosity.append(0)
#             else:
#                 sinuosity.append(dist_sum / dist_line)

        features.append(np.concatenate([get_feature(speed_list),
                                        angle_feature[:1], turn_feature[:1]]))

    return features

In [6]:
X_train_tsfresh, y_train = feature_generate_tsfresh()

Index(['lat__quantile__q_0.9', 'lon__quantile__q_0.1', 'lat__maximum',
       'lon__minimum', '速度__number_crossing_m__m_1', 'lat__quantile__q_0.8',
       'lon__quantile__q_0.2', 'lat__minimum', 'lon__quantile__q_0.4',
       'lon__maximum', '速度__quantile__q_0.7',
       '速度__ratio_beyond_r_sigma__r_0.5',
       '方向__fft_coefficient__coeff_5__attr_"real"',
       '速度__ar_coefficient__k_10__coeff_1', 'lat__number_cwt_peaks__n_1',
       'lon__approximate_entropy__m_2__r_0.1',
       'lon__fft_coefficient__coeff_64__attr_"angle"', 'lon__quantile__q_0.9',
       'lat__quantile__q_0.3', 'lat__fft_coefficient__coeff_5__attr_"real"',
       'lon__quantile__q_0.3',
       'lon__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',
       'lat__fft_coefficient__coeff_74__attr_"angle"', 'lon__quantile__q_0.8',
       '速度__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'lat__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_2',
       'lat__quantile__q_0.2', 'lon__fft_coefficient_

/Users/yitinglin/Projects/fishing-classification/env/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


(8166, 50) (8166, 33)


In [7]:
X_paper_train = get_paper_feature()

/Users/yitinglin/Projects/fishing-classification/env/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/yitinglin/Projects/fishing-classification/env/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/Users/yitinglin/Projects/fishing-classification/env/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/yitinglin/Projects/fishing-classification/env/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [8]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

In [9]:
from sklearn.impute import SimpleImputer

X_train = np.concatenate([X_train_tsfresh.values, np.array(X_paper_train)], axis=1)

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imputer.fit_transform(pd.DataFrame(X_train).replace([np.inf, -np.inf], np.nan).values)

In [10]:
X_train.shape, X_train_tsfresh.shape

((8166, 70), (8166, 56))

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
kf = KFold(n_splits=5, random_state=2019, shuffle=True)
model_v1_list = []
score_v1_list = []
for train_index, test_index in kf.split(X_train):
    model_v1 = get_model()
    eval_set = (X_train[test_index], y_train[test_index])
    model_v1.fit(X_train[train_index], y_train[train_index])
    model_v1_list.append(model_v1)
    score_v1_list.append(f1_score(y_train[test_index], model_v1.predict(X_train[test_index]), average='macro'))
print(score_v1_list)
print(np.mean(score_v1_list), np.std(score_v1_list))

kf = KFold(n_splits=5, random_state=22, shuffle=True)
model_v2_list = []
score_v2_list = []
for train_index, test_index in kf.split(X_train):
    model_v2 = get_model_v2()
    eval_set = (X_train[test_index], y_train[test_index])
    model_v2.fit(X_train[train_index], y_train[train_index])
    model_v2_list.append(model_v2)
    score_v2_list.append(f1_score(y_train[test_index], model_v2.predict(X_train[test_index]), average='macro'))
print(score_v2_list)
print(np.mean(score_v2_list), np.std(score_v2_list))

[0.9070596897760149, 0.9034521057812653, 0.909143147035223, 0.9101786452238425, 0.9101814858807319]
0.9080030147394156 0.0025447844899796875
[0.9029012439218346, 0.9235586535639501, 0.902513600024824, 0.9110527876200992, 0.9008766875083026]
0.9081805945278021 0.00846247741675804


In [13]:
kf = KFold(n_splits=5, random_state=22, shuffle=True)
model_v3_list = []
score_v3_list = []
for train_index, test_index in kf.split(X_train):
    model_v3 = get_model_v3()
    eval_set = (X_train[test_index], y_train[test_index])
    model_v3.fit(X_train[train_index], y_train[train_index])
    
    train_proba = model_v3.predict_proba(X_train[train_index])
    test_proba = model_v3.predict_proba(X_train[test_index])

    model_v3_list.append(model_v3)
    score_v3_list.append(f1_score(y_train[test_index], model_v3.predict(X_train[test_index]), average='macro'))

print(score_v3_list)
print(np.mean(score_v3_list), np.std(score_v3_list))

[0.9041714371155161, 0.9270159183514962, 0.9082592533329926, 0.9127207010997207, 0.8988635270078261]
0.9102061673815103 0.009568223887799786


In [14]:
kf = KFold(n_splits=5, random_state=22, shuffle=True)
model_v4_list = []
score_v4_list = []
for train_index, test_index in kf.split(X_train):
    model_v4 = get_model_v4()
    eval_set = (X_train[test_index], y_train[test_index])
    model_v4.fit(X_train[train_index], y_train[train_index])
    
    train_proba = model_v4.predict_proba(X_train[train_index])
    test_proba = model_v4.predict_proba(X_train[test_index])

    model_v4_list.append(model_v4)
    score_v4_list.append(f1_score(y_train[test_index], model_v4.predict(X_train[test_index]), average='macro'))

print(score_v4_list)
print(np.mean(score_v4_list), np.std(score_v4_list))

[0.9049565032682895, 0.9294512892332593, 0.9184542968671158, 0.9065878596411672, 0.8987416682891181]
0.9116383234597899 0.010956613195528455


In [15]:
df = pd.DataFrame(X_train)
df['type'] = le.inverse_transform(y_train)

In [16]:
df.to_csv('./train.csv', index=False)

In [18]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

def get_model_v5():
    exported_pipeline = make_pipeline(
        StackingEstimator(estimator=BernoulliNB(alpha=1.0, fit_prior=False)),
        RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.45, n_estimators=100), step=0.6500000000000001),
        RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=100)
    )
    # Fix random state for all the steps in exported pipeline
    set_param_recursive(exported_pipeline.steps, 'random_state', 42)
    
    return exported_pipeline

kf = KFold(n_splits=5, random_state=1, shuffle=True)
model_v5_list = []
score_v5_list = []
for train_index, test_index in kf.split(X_train):
    model_v5 = get_model_v5()
    eval_set = (X_train[test_index], y_train[test_index])
    model_v5.fit(X_train[train_index], y_train[train_index])
    
    train_proba = model_v5.predict_proba(X_train[train_index])
    test_proba = model_v5.predict_proba(X_train[test_index])

    model_v5_list.append(model_v5)
    score_v5_list.append(f1_score(y_train[test_index], model_v5.predict(X_train[test_index]), average='macro'))

print(score_v5_list)
print(np.mean(score_v5_list), np.std(score_v5_list))

[0.9080022218336543, 0.9109605244052391, 0.9171413791194124, 0.9053365966488025, 0.9144011732928566]
0.9111683790599929 0.0042474728837917485


In [23]:
proba = pd.read_csv('./probaresult.csv', header=None, index_col=0)


,1,2,3
0,,,
28165,0.006352,0.980176,0.013472


In [29]:
(model_v5.predict_proba([X_train[test_index][0]]) + proba.values) / 2

array([[0.43817593, 0.54008796, 0.02173611]])